In [1]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Input
from keras.layers import Bidirectional
from ray import tune
from ray.tune.registry import register_env
from keras.models import Model

In [19]:
sequence_length=3
state_len = 52
num_action = 1
encoding_len = state_len + num_action
NUM_NODES = 6
NODE_CLASSES = [3, 4]

class WorldMovelEnv(gym.Env):

    def __init__(self):
        
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(42,))
        self.action_space = gym.spaces.Discrete(20)
        
        self.step_count = 0
        '''
        #Afterstate Model
        self.as_model = Sequential()
        self.as_model.add(Input(shape=(42+20,)))
        self.as_model.add(Dense(512, activation='relu'))
        self.as_model.add(Dense(512, activation='relu'))
        self.as_model.add(Dense(512, activation='relu'))
        self.as_model.add(Dense(42, activation='sigmoid'))
        #self.as_model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])
        self.as_model.load_weights('AfterStateModel')
        
        #Next State Model
        self.ns_model = Sequential()
        self.ns_model.add(Input(shape=(42+20,)))
        self.ns_model.add(Dense(512, activation='relu'))
        self.ns_model.add(Dense(512, activation='relu'))
        self.ns_model.add(Dense(512, activation='relu'))
        self.ns_model.add(Dense(42, activation='sigmoid'))
        #self.ns_model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])
        self.ns_model.load_weights('NextStateModel')
        '''
        #Reward Model
        self.r_model = Sequential()
        self.r_model.add(Input(shape=(43+20,)))
        self.r_model.add(Dense(512, activation='relu'))
        self.r_model.add(Dense(512, activation='relu'))
        self.r_model.add(Dense(512, activation='relu'))
        self.r_model.add(Dense(6, activation='softmax'))
        #self.r_model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])
        self.r_model.load_weights('RewardModel')
        
        input_ = Input(shape=(43+20,))
        x = Dense(512, activation='relu')(input_)
        x = Dense(512, activation='relu')(x)
        outs = []
        for i in range(NUM_NODES):
            for n in NODE_CLASSES:
                x_ = Dense(128, activation='relu')(x)
                outs.append(Dense(n, activation='softmax', name=str(i)+str(n))(x_))

        self.ns_model_multi_model = Model(input_, outs)
        self.ns_model_multi_model.load_weights('AfterStateModel_MulitLabel')
        
        self.reward_map = np.load('reward_map.npy', allow_pickle=True).item()
        self.state = np.array([0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1.])
        self.action = np.zeros(20)

    def step(self, action):
        self.step_count += 1
        
        #self.state_action[0,52] = action / 145
        #obs_index_probs = self.st_model.predict(np.array([self.state_action]))
        #obs_index = np.argmax(obs_index_probs)
        #obs_index = np.random.choice(np.arange(9674), p=obs_index_probs[0])
        #new_state = self.index_to_state[obs_index]
        #new_state = np.frombuffer(new_state)
        
        
        #for i in range(1,3):
        #    self.state[i,:] = self.state[i-1,:]
        
        self.action = np.zeros(20)
        self.action[action] = 1
        state_action = np.concatenate([self.state, [self.step_count/100], self.action])
        #afterstate_probs = self.as_model.predict(np.array([state_action]), verbose=0)
        #afterstate = np.array(np.random.rand(1, 42) < afterstate_probs, dtype=np.int8)
        
        #while True:
        #    new_state_probs = self.ns_model.predict(np.array([state_action]), verbose=0)
        #    new_state = np.array(np.random.rand(1, 42) < new_state_probs, dtype=np.int8)
        #    if int(np.sum(self.state-new_state)) == 0:
        #        break
        #self.state = new_state[0]
        probs = self.ns_model_multi_model.predict(np.array([state_action]), verbose=0)
        self.state = np.zeros(42)
        index_state = 0; index = 0
        for i in range(NUM_NODES):
            for n in NODE_CLASSES:
                self.state[index_state+np.random.choice(np.arange(n), p=probs[index][0])] = 1
                index_state += n; index += 1
        
       # reward_probs = self.r_model.predict(np.array([self.state]), verbose=0)
        #reward = np.random.choice(np.arange(6), p=reward_probs[0])
        #reward = self.reward_map[reward]
        
        #for i in range(1,3):
        #    self.state_action[i,:] = self.state_action[i-1,:]
        #
        #self.state_action[0,:] = np.concatenate([new_state, [0]])
        
        reward_probs = self.r_model.predict(np.array([state_action]))
        reward_index = np.random.choice(np.arange(6), p=reward_probs[0])
        reward = self.reward_map[reward_index]
        
        done = self.step_count == 99
        if done:
            self.step_count = 0
        return self.state, reward, done, {}

    def reset(self):
        step_count = 0
        
        #self.state_action = np.zeros((sequence_length,encoding_len))
        #self.state = np.zeros((sequence_length,state_len))
        
        #obs_index_probs = self.st_model.predict(np.array([self.state_action]))
        #obs_index = np.argmax(obs_index_probs)
        #obs_index = np.random.choice(np.arange(9674), p=obs_index_probs[0])
        #new_state = self.index_to_state[obs_index]
        #new_state = np.frombuffer(new_state)
        self.state = np.array([0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
           0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
           1., 0., 0., 1., 0., 0., 0., 1.])
      
        return self.state
    

    def render(self, mode='human', close=False):
        pass

    def close(self):
        pass
    
def env_creator(config):
    return WorldMovelEnv() 

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
test = WorldMovelEnv()
test.reset()
for i in range(100):
    print(test.step(np.random.randint(20)))

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1/1 [==============================] - 0s 107ms/step
(array([0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1.]), 0.0, False, {})
1/1 [==============================] - 0s 38ms/step
(array([0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 1.]), 0.0, False, {})
1/1 [==============================] - 0s 43ms/step
(array([0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 1.]), 0.0, False, {})
1/1 [==============================] - 0s 34ms/step
(array([0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 0


KeyboardInterrupt



In [21]:
register_env(name="DreamCybORG", env_creator=env_creator)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

In [32]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec

config = (
    PPOConfig()
    #Each rollout worker uses a single cpu
    .rollouts(num_rollout_workers=20, num_envs_per_worker=1, horizon=100)\
    .training(sgd_minibatch_size = 100, train_batch_size=2000, gamma=0.99, lr=0.00005, 
              model={"fcnet_hiddens": [256, 256], "fcnet_activation": "tanh",})\
    .environment(disable_env_checking=True, env = 'DreamCybORG')\
    .resources(num_gpus=1)\
    .framework('torch')\
    #.exploration(explore=True, exploration_config={"type": "RE3", "embeds_dim": 128, "beta_schedule": "constant", "sub_exploration": {"type": "StochasticSampling",},})\
)
trainer = config.build()


/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
(pid=24384) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=24384)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=24394) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=24394)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=24436) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Versi

(RolloutWorker pid=24991) WARNING:tensorflow:OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.
(RolloutWorker pid=24992) WARNING:tensorflow:OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.
(RolloutWorker pid=24987) WARNING:tensorflow:OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.
(RolloutWorker pid=24989) WARNING:tensorflow:OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.
(RolloutWorker pid=24988) WARNING:tensorflow:OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.
2023-02-08 14:53:57,690	INFO trainable.py:172 -- Trainable.setup took 14.686 seconds. If your trainable is slow to initialize, con

In [46]:
for i in range(500):
    print_results(trainer.train())

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


 307 	r_mean: -51.3 	r_max: -32.8 	r_min: -73.4
 308 	r_mean: -52.7 	r_max: -33.5 	r_min: -73.4
 309 	r_mean: -53.1 	r_max: -33.5 	r_min: -73.4
 310 	r_mean: -52.0 	r_max: -35.5 	r_min: -73.4
 311 	r_mean: -52.4 	r_max: -35.5 	r_min: -73.3
 312 	r_mean: -51.3 	r_max: -33.4 	r_min: -73.3
 313 	r_mean: -50.9 	r_max: -33.4 	r_min: -74.4
 314 	r_mean: -49.9 	r_max: -32.6 	r_min: -74.4
 315 	r_mean: -49.8 	r_max: -32.6 	r_min: -74.4
 316 	r_mean: -48.5 	r_max: -32.6 	r_min: -74.4
 317 	r_mean: -48.7 	r_max: -32.6 	r_min: -74.4
 318 	r_mean: -48.5 	r_max: -30.8 	r_min: -69.8
 319 	r_mean: -49.2 	r_max: -30.8 	r_min: -72.7
 320 	r_mean: -49.7 	r_max: -30.7 	r_min: -72.7
 321 	r_mean: -51.2 	r_max: -30.7 	r_min: -72.7
 322 	r_mean: -51.8 	r_max: -30.7 	r_min: -72.7


2023-02-08 16:41:05,748	ERROR actor_manager.py:486 -- Ray error, taking actor 16 out of service. The actor died unexpectedly before finishing this task.
2023-02-08 16:41:05,752	ERROR actor_manager.py:486 -- Ray error, taking actor 20 out of service. The actor died unexpectedly before finishing this task.


 323 	r_mean: -52.1 	r_max: -30.7 	r_min: -72.7


2023-02-08 16:41:07,754	ERROR actor_manager.py:486 -- Ray error, taking actor 15 out of service. The actor died unexpectedly before finishing this task.


 324 	r_mean: -50.9 	r_max: -34.8 	r_min: -73.6


(raylet) [2023-02-08 16:41:41,519 E 17097 17097] (raylet) node_manager.cc:3097: 45 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 0eb798f5450c6700c1f813c16d9fc4b860c81125e5dd19ddab711062, IP: 192.168.224.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.224.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.

KeyboardInterrupt



In [7]:
trainer.load_checkpoint('results/checkpoint_000268')

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
for i in range(10):

    cyborg = CybORG(path, 'sim', agents={'Red': RedMeanderAgent})
    wrapped_cyborg = wrap(cyborg)

    observation = wrapped_cyborg.reset()
    wrapped_cyborg.reset()
    obs = wrapped_cyborg.step(5)[3]['observation']
    print(obs)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'CybORG' is not defined

In [51]:
test.reset()
obs2, r, d, i = test.step(5)
obs - obs2

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1/1 [==============================] - 0s 35ms/step


array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

In [45]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os
from tqdm import trange

MAX_EPS = 100
agent_name = 'Blue'

def wrap(env):
    return RLlibWrapper(agent_name="Blue", env=env)

def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2Small.yaml'
    agents = {"Red": RedMeanderAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

register_env(name="CybORG", env_creator=env_creator)


path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario2Small.yaml'
obs = []
#print(f'using CybORG v{cyborg_version}, {scenario}\n')
for red_agent in [RedMeanderAgent]:#, RedMeanderAgent]:

    cyborg = CybORG(path, 'sim', agents={'Red': red_agent})
    wrapped_cyborg = wrap(cyborg)

    observation = wrapped_cyborg.reset()
    obs.append(observation)
    # observation = cyborg.reset().observation

    action_space = wrapped_cyborg.get_action_space(agent_name)
    # action_space = cyborg.get_action_space(agent_name)
    total_reward = []
    actions = []
    for i in trange(MAX_EPS):
        r = []
        #a = []
        cyborg = CybORG(path, 'sim', agents={'Red': red_agent})
        wrapped_cyborg = wrap(cyborg)

        observation = wrapped_cyborg.reset()
        obs.append(observation)
        # observation = cyborg.reset().observation

        # cyborg.env.env.tracker.render()
        for j in range(100):
            
            
            action = trainer.compute_single_action(observation, explore=False)
            #action_vec = np.zeros(145)
            #action_vec[int(action)] = 1
            #action = agent.get_action(observation, action_space)
            observation, rew, done, info = wrapped_cyborg.step(action)
            obs.append(observation)
            #actions.append(action_vec)
            # result = cyborg.step(agent_name, action)
            r.append(rew)
            # r.append(result.reward)
           # a.append((str(cyborg.get_last_action('Blue')), str(cyborg.get_last_action('Red'))))
        total_reward.append(sum(r))
        # observation = cyborg.reset().observation
        observation = wrapped_cyborg.reset()
    print(f'Average reward for red agent {red_agent.__name__} and steps {100} is: {mean(total_reward):.1f} with a standard deviation of {stdev(total_reward):.1f}')
    #return mean(total_reward), np.mean(np.array(obs), axis=0),  np.mean(np.array(actions), axis=0)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 100/100 [06:58<00:00,  4.18s/it]

Average reward for red agent RedMeanderAgent and steps 100 is: -34.7 with a standard deviation of 4.0


In [55]:
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.policy.policy import PolicySpec

config = (
    DQNConfig()
    #Each rollout worker uses a single cpu
    .rollouts(num_rollout_workers=10, num_envs_per_worker=1, horizon=100)\
    .training(gamma=0.99, lr=0.005, 
              model={"fcnet_hiddens": [256, 256], "fcnet_activation": "tanh",})\
    .environment(disable_env_checking=True, env = 'DreamCybORG')\
    .resources(num_gpus=1)\
    .framework('torch')\
    #.exploration(explore=True, exploration_config={"type": "RE3", "embeds_dim": 128, "beta_schedule": "constant", "sub_exploration": {"type": "StochasticSampling",},})\
)
trainer = config.build()
for i in range(500):
    print_results(trainer.train())

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
(pid=25909) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=25909)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=25826) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=25826)   if (distutils.version.LooseVersion(tf.__version__) <
(pid=25829) /usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Versi

2023-02-08 12:36:23,787	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
2023-02-08 12:36:26,890	WARNING deprecation.py:47 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!


   1 	r_mean: -81.2 	r_max: -67.7 	r_min: -97.9


/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/algorithm.py:1716: DeprecationWarning: invalid escape sequence \|
  """Adds a new policy to this Algorithm.
/usr/local/lib/python3.8/dist-packages/ray/rllib/algorithms/dqn/dqn.py:54: DeprecationWarning: invalid escape sequence \ 
  """Defines a configuration class from which a DQN Algorithm can be built.

KeyboardInterrupt



In [ ]:
test = WorldMovelEnv()
import time 

t = time.time()
print(test.reset())
for i in range(100):
    print(test.step(55))
print(t-time.time())

In [14]:
trainer.save('results')

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'results/checkpoint_000268'

In [9]:
!pip install tqdm

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:281: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     |████████████████████████████████| 78 kB 3.5 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
